In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

=====> /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12


File /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);



--- Translating Stan model to C++ code ---
bin/stanc  --o=/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.hpp /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan
Model name=m12_6_model
Input file=/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan
Output file=/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.hpp
clang++ -std=c++1y -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare      -O3 -I src -I stan/src -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.3 -I stan/lib/stan_math/lib/boost_1.66.0 -I stan/lib/stan_math/lib/sundials_3.1.0/include    -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -DBOOST_PHOENIX_NO_VARIADIC_EXPRESSION     -c -MT /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.o -MT /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6 -include /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.h

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE       MCSE         ESS    
         lp__ -39.547584700  3.0695736406 0.04853422075 0.1168004118  690.664869
accept_stat__   0.926867053  0.0968584820 0.00153146707 0.0017268753 1000.000000
   stepsize__   0.056565925  0.0018793574 0.00002971525 0.0003009004   39.009752
  treedepth__   5.761750000  0.4884122993 0.00772247652 0.0135110947 1000.000000
 n_leapfrog__  65.072000000 27.8187758011 0.43985346625 0.5555855308 1000.000000
  divergent__   0.000000000  0.0000000000 0.00000000000 0.0000000000         NaN
     energy__  46.047208425  4.0218982970 0.06359179568 0.1485520564  733.001447
        alpha   1.095902219  0.7564406991 0.01196037762 0.0235354830 1000.000000
  a_society.1  -0.202671782  0.2462745662 0.00389394279 0.0055250586 1000.000000
  a_society.2   0.045578240  0.2134544942 0.00337501189 0.0050349721 1000.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*